# Web & Scraping

Requires: urllib, requests, socket, re, lxml, io, bs4, scrapy, sqlite3, pandas.

План:
* 101 101
* HTML 101
* HTTP + REST 101
* Web scraping

In [1]:
# Убедитесь, что вотэтовотвсё установлено
import urllib, requests, socket, re, lxml, io, bs4, scrapy, sqlite3, pandas, sqlalchemy

Заметка на полях:
# 101 101

В американской системе образования курсы имеют индекс, например, [CS101](https://lagunita.stanford.edu/courses/Engineering/CS101/Summer2014/about).

101 -- это вводный курс в какую-то тему, просто так повелось. Intro.


# Интернет 101

Что происходит, когда вы вбиваете в поисковике адрес и жмете Enter?

URL:  
`<схема>:[//[<логин>[:<пароль>]@]<хост>[:<порт>]][/<URL‐путь>][?<параметры>][#<якорь>]`  
Например:  
`https://pp.userapi.com/c834101/v834101778/13450d/9yxFBjsPxN8.jpg`

Вы выступаете **клиентом**

1. **DNS**: URL -> IP-адрес и порт **сервера** (87.240.129.71:443)
2. **HTTP(S)**: IP + URL-путь + параметры -> **ресурс** (текст (в т.ч. *HTML-документ*), картинка, звук и т.д.)
3. **Движок браузера** пытается отрисовать документ
4. *Скорее всего*, внутри документа написано, что ему требуются еще какие-то ресурсы, у них есть URI, т.е. возвращаемся к п.1
5. Пока все это происходит, (и после того, как произошло), выполняется **JavaScript**-код

# HTML 101

HyperText Markup Language

HTML -- язык разметки, который используется для web-страниц.

Это механизм для получения структурного текста, который понимают браузеры.

Структура в тексте задаётся вложенными тегами, теги определяют то, как текст будет показан (отрендерен).

Это тег: `<тег>`, теги бывают открывающими (`<тег>`) и закрывающими (`</тег>`).

Пример HTML-разметки:

```html
<!DOCTYPE html>
<html>
   <head>
      <meta charset="utf-8" />
      <title>HTML Document</title>
   </head>
   <body>
      <p> <!-- p -- это параграф, а в такие странные скобки заключается комментарий -->
         <b>
            Этот текст будет полужирным, <i>а этот — ещё и курсивным</i>.
         </b>
      </p>
   </body>
</html>
```

Так он выглядит после рендеринга:

<!DOCTYPE html>
<html>
   <head>
      <meta charset="utf-8" />
      <title>HTML Document</title>
   </head>
   <body>
      <p> <!-- p -- это параграф, а в такие странные скобки заключается комментарий -->
         <b>
            Этот текст будет полужирным, <i>а этот — ещё и курсивным</i>.
         </b>
      </p>
   </body>
</html>

Видно, что HTML-разметка имеет древовидную структуру:<br>
каждый тег (вершина дерева) имеет 0 (тогда это лист дерева) или больше (тогда это внутренняя вершина) вложенных в него тегов.

Значит, чтобы доставать из HTML какую-то информацию, можно использовать его структуру.

Мы этим займёмся чуть позже.

### P.S. 
На самом деле, такую же структуру имеет и формат XML (другой язык разметки).

Формально, HTML -- это более стандартизированное подмножество XML.

### P.P.S.

А ещё у тегов бывают **атрибуты**:

```html
<a href="http://example.com">Ссылка на example.com</a>
```
Отрендерим этот кусок:
<a href="http://example.com">Ссылка на example.com</a>

### P.P.P.S.

А этот текст (безусловно, имеющий структуру), был написан с помощью другого языка разметки -- **Markdown**.

## HTTP 101

HyperText Transfer Protocol

Протокол передачи гипертекста:
* **Протокол** -- набор логических соглашений о взаимодействии программ, стандарт;
* **передача** -- мы живём в интернете, т.е. мир не ограничивается нашим компьютером => информацию надо передавать;
* **гипертекст** -- HTML, например.

Основная идея: все компьютеры делятся на **сервера** (те, что хранят информацию) и **клиенты** (те, что запрашивают её).

На самом деле, в HTTP мы передаём не только гипертекст, а **ресурсы** -- философское обобщение, куда попадает и гипертекст, и картинки, и музыка и т.д. Но стандарт подходит для всего этого.

Каждый ресурс имеет адрес -- **URI** (Uniform Resource Identifier) -- аналог пути в системе компьютера.

Не будем говорить про низкоуровневую сторону вопроса: стек TCP/IP, сокеты, порты и т.д.

## Структура HTTP

Чтобы получить данные, клиент делает запрос на сервер. В запросе должны быть 3 части:
* Starting line -- определяет тип сообщения;
* Headers -- характеризуют тело сообщения, параметры передачи и прочие сведения;
* Message body -- непосредственно данные сообщения.

Ответное сообщение от сервера имеет такую же структуру.

Чтобы посылать эти запросы "руками", а не через браузер, можно использовать утилиту `curl`. Ей и воспользуемся.

In [2]:
!curl example.com/index.html -v

*   Trying 93.184.216.34:80...
* Connected to example.com (93.184.216.34) port 80 (#0)
> GET /index.html HTTP/1.1
> Host: example.com
> User-Agent: curl/7.71.1
> Accept: */*
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< Accept-Ranges: bytes
< Age: 23352
< Cache-Control: max-age=604800
< Content-Type: text/html; charset=UTF-8
< Date: Thu, 01 Apr 2021 23:09:52 GMT
< Etag: "3147526947"
< Expires: Thu, 08 Apr 2021 23:09:52 GMT
< Last-Modified: Thu, 17 Oct 2019 07:18:26 GMT
< Server: ECS (bsa/EB1E)
< Vary: Accept-Encoding
< X-Cache: HIT
< Content-Length: 1256
< 
<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemF

## Методы HTTP

HTTP метод определяет операцию, которую мы хотим произвести над ресурсом.

Самые частые методы: GET, POST, PUT, DELETE (но есть и другие).

* **GET** -- запрос содержимого ресурса. <br>GET-запросы идемпотентны. Поэтому они могут кэшироваться.
* **POST** -- передача данных на ресурс (например, при отправке комментария на форуме или вводе пароля на сайте). <br> Не идемпотентны => при отправке одного и того же комментария на форум он появится там дважды.  <br>Не кэшируются.
* **PUT** -- передача данных в конкретный URI (изменение существующего ресурса). Не кэшируются.
* **DELETE** -- удаление ресурса.

## Коды HTTP

В ответном сообщении придёт код ответа HTTP, который определяет результат выполнения операции.

Самые частые коды: `200 OK`, `400 BadRequest`, `404 Not Found`, `500 Internal Server Error`.

Общий обзор кодов:
* **1xx** -- Informational. Информационные коды, например, `102 Processing` (запрос пока обрабатывается);
* **2xx** -- Success. Успех. Всё хорошо, запрос отработал и ничего не сломал. По крайней мере, пока.
* **3xx** -- Redirection. Перенаправление на другой ресурс/страницу.
* **4xx** -- Client error. Ошибка клиента (неверные данные запроса или неправильный путь).
* **5xx** -- Server error. Что-то сломалось на сервере (там поделили на ноль, например).

## Разбор примера

Давайте ещё раз посмотрим на запрос к example.com/index.html.

Служебная информация от `curl`:

```
*   Trying 2606:2800:220:1:248:1893:25c8:1946...
*   Trying 93.184.216.34...
* Connected to example.com (2606:2800:220:1:248:1893:25c8:1946) port 80 (#0)
```
Запрос клиента:
```
> GET /index.html HTTP/1.1    # Starting line: метод GET, URI -- /index.html, версия протокола -- HTTP/1.1
> Host: example.com           # Заголовки сообщения
> User-Agent: curl/7.47.0
> Accept: */*
>                             # Пустое тело, т.к. мы ничего не передали на сервер.
```

Ответ сервера:
```
< HTTP/1.1 200 OK                          # Starting line: версия протокола и код ответа
< Cache-Control: max-age=604800            # Заголовки ответа
< Content-Type: text/html; charset=UTF-8
< Date: Tue, 19 Mar 2019 21:25:21 GMT
< Etag: "1541025663+gzip+ident"
< Expires: Tue, 26 Mar 2019 21:25:21 GMT
< Last-Modified: Fri, 09 Aug 2013 23:54:35 GMT
< Server: ECS (bsa/EB1E)
< Vary: Accept-Encoding
< X-Cache: HIT
< Content-Length: 1270
<                                          # Пустая строка -- нужна по стандарту
<!doctype html>                            # Тело ответа -- HTML-документ
<html>
<head>
    <title>Example Domain</title>
. . .
```


## REST

 <a href="https://en.wikipedia.org/wiki/Representational_state_transfer">Representational State Transfer</a> 

Набор дополнительных ограничений на HTTP, указывающий, как правильно работать с методами, кодами и т.д.

Можно назвать его аналогом pep для Питона.

# Web scraping. Практика!

* Получение html
* Парсинг html

## Получение html

Варианты инструментов:

* urllib 
* requests (de-facto standard)
* socket (low-level)



### urllib 

In [3]:
import urllib.request

In [4]:
response = urllib.request.urlopen('http://example.com/')
html = response.read()
print(html)

b'<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: #fdfdff;\n        border-radius: 0.5em;\n        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700px) {\n        div {\n            margin: 0 auto;\n            width: auto;\n        }\n    }\n    </style>    \n</head>\n\n<body>\n<div>\n    

In [5]:
html = html.decode('utf-8')
print(html)

<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This domai

На всякий случай сохраним этот html, пригодится.

In [6]:
with open('example.com.txt', 'w', encoding='utf-8') as f:
    f.write(html)

А что, кроме html?

In [7]:
print(dir(response))

['__abstractmethods__', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_abc_impl', '_checkClosed', '_checkReadable', '_checkSeekable', '_checkWritable', '_check_close', '_close_conn', '_get_chunk_left', '_method', '_peek_chunked', '_read1_chunked', '_read_and_discard_trailer', '_read_next_chunk_size', '_read_status', '_readall_chunked', '_readinto_chunked', '_safe_read', '_safe_readinto', 'begin', 'chunk_left', 'chunked', 'close', 'closed', 'code', 'debuglevel', 'detach', 'fileno', 'flush', 'fp', 'getcode', 'getheader', 'getheaders', 'geturl', 'headers', 'info', 'isatty', 'isclosed', 'length', 'msg', 'peek', 'read', 'read1', 'readable', 'readinto', 

Всё, что есть в HTTP-сообщении.

In [8]:
print(response.url)
print(response.msg)
print(response.code)
print('Заголовки: \n{}'.format(response.headers))
# . . .

http://example.com/
OK
200
Заголовки: 
Age: 442091
Cache-Control: max-age=604800
Content-Type: text/html; charset=UTF-8
Date: Thu, 01 Apr 2021 23:09:53 GMT
Etag: "3147526947+ident"
Expires: Thu, 08 Apr 2021 23:09:53 GMT
Last-Modified: Thu, 17 Oct 2019 07:18:26 GMT
Server: ECS (bsa/EB17)
Vary: Accept-Encoding
X-Cache: HIT
Content-Length: 1256
Connection: close




### requests
HTTP for Humans

In [9]:
import requests

In [10]:
response = requests.get('http://example.com')
response

<Response [200]>

In [11]:
print(dir(response))

['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', '_next', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'next', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']


In [12]:
print(response.url)
print(response.connection)
print(response.headers)
print(response.ok)
print(response.status_code)
print(response.encoding)
print(response.links)

http://example.com/
{'Content-Encoding': 'gzip', 'Accept-Ranges': 'bytes', 'Age': '455261', 'Cache-Control': 'max-age=604800', 'Content-Type': 'text/html; charset=UTF-8', 'Date': 'Thu, 01 Apr 2021 23:09:53 GMT', 'Etag': '"3147526947"', 'Expires': 'Thu, 08 Apr 2021 23:09:53 GMT', 'Last-Modified': 'Thu, 17 Oct 2019 07:18:26 GMT', 'Server': 'ECS (bsa/EB15)', 'Vary': 'Accept-Encoding', 'X-Cache': 'HIT', 'Content-Length': '648'}
True
200
UTF-8
{}


In [13]:
print(response.text)

<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This domai

### socket (low-level, если вам в жизни мало ассемблера)

In [14]:
import socket

In [15]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = socket.gethostbyname('www.example.com')
port = 80

sock.connect((host,port))
sock.sendall(b'GET / HTTP/1.1\r\nHost: www.example.com\r\n\r\n')

val = sock.recv(4096)
print(val.decode('utf-8'))

HTTP/1.1 200 OK
Age: 455262
Cache-Control: max-age=604800
Content-Type: text/html; charset=UTF-8
Date: Thu, 01 Apr 2021 23:09:54 GMT
Etag: "3147526947+ident"
Expires: Thu, 08 Apr 2021 23:09:54 GMT
Last-Modified: Thu, 17 Oct 2019 07:18:26 GMT
Server: ECS (bsa/EB15)
Vary: Accept-Encoding
X-Cache: HIT
Content-Length: 1256

<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2p

In [16]:
# Split off the HTTP headers
val = val.split(b'\r\n\r\n',1)[1]
print(val.decode('utf-8'))

<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This domai

## Парсинг html

Варианты инструментов:
* re
* lxml
* BeautifulSoup

In [17]:
with open('example.com.txt', 'r', encoding='utf-8') as f:
    html = f.read()
print(html)

<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This domai

### re

Давайте забудем, что HTML -- дерево, и попробуем попарсить его, как строчку.

In [18]:
import re

In [19]:
h1 = re.findall(r'<h1>[\w ]+</h1>', html)
print(h1)
h1 = re.findall(r'<h1>([\w ]+)</h1>', html)
print(h1)

['<h1>Example Domain</h1>']
['Example Domain']


In [20]:
paragraphs = re.findall(r'<p>(.*)</p>', html)
paragraphs

['<a href="https://www.iana.org/domains/example">More information...</a>']

Там же 2 параграфа, почему не нашёлся второй?


In [21]:
paragraphs = re.findall(r'<p>([\w\W]*)</p>', html) 
paragraphs

['This domain is for use in illustrative examples in documents. You may use this\n    domain in literature without prior coordination or asking for permission.</p>\n    <p><a href="https://www.iana.org/domains/example">More information...</a>']

Опять плохо..

In [22]:
paragraphs = re.findall(r'<p>([\w\W]*?)</p>', html) # * - greedy; *? - lazy (non-greedy) 
paragraphs

['This domain is for use in illustrative examples in documents. You may use this\n    domain in literature without prior coordination or asking for permission.',
 '<a href="https://www.iana.org/domains/example">More information...</a>']

Жесть, да?

### lxml

Воспользуемся библиотекой, которая знает про структуру XML (и HTML).

In [23]:
from lxml import etree
from io import StringIO, BytesIO

In [24]:
parser = etree.HTMLParser()
tree = etree.parse(StringIO(html), parser)
tree

In [25]:
print(dir(tree))

['__class__', '__copy__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_setroot', 'docinfo', 'find', 'findall', 'findtext', 'getelementpath', 'getiterator', 'getpath', 'getroot', 'iter', 'iterfind', 'parse', 'parser', 'relaxng', 'write', 'write_c14n', 'xinclude', 'xmlschema', 'xpath', 'xslt']


In [26]:
print(tree.getroot())
print(etree.tostring(tree.getroot(), pretty_print=True, method='html'))

<Element html at 0x7f95af7e0500>
b'<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8">\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: #fdfdff;\n        border-radius: 0.5em;\n        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700px) {\n        div {\n            margin: 0 auto;\n            width: auto;\n        }\n    }\n    </style>    \n</head>\n\n<body>\n<

In [27]:
paragraphs = tree.xpath('//p')
for p in paragraphs:
    print(p.text)  

This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.
None


In [28]:
for p in paragraphs:
    print(etree.tostring(p, pretty_print=True, method='html'))  

b'<p>This domain is for use in illustrative examples in documents. You may use this\n    domain in literature without prior coordination or asking for permission.</p>\n    \n'
b'<p><a href="https://www.iana.org/domains/example">More information...</a></p>\n\n'


In [29]:
hrefs = tree.xpath('//a')
for href in hrefs:
    print(href.text)  
    print(href.attrib)  

More information...
{'href': 'https://www.iana.org/domains/example'}


In [30]:
specific_hrefs = tree.xpath('//a[@href="https://www.iana.org/domains/example"]')
specific_hrefs

[<Element a at 0x7f95af858b40>]

### BeautifulSoup

Ещё один вариант, который хорошо подходит для парсинга HTML.

In [31]:
# try:
from bs4 import BeautifulSoup
# except:
#     !pip install bs4 

In [32]:
soup = BeautifulSoup(html, 'lxml')

In [33]:
paragraphs = soup.find_all('p')
paragraphs

[<p>This domain is for use in illustrative examples in documents. You may use this
     domain in literature without prior coordination or asking for permission.</p>,
 <p><a href="https://www.iana.org/domains/example">More information...</a></p>]

In [34]:
hrefs = soup.find_all('a')
hrefs

[<a href="https://www.iana.org/domains/example">More information...</a>]

In [35]:
hrefs = soup.find_all('a', href='https://www.iana.org/domains/example')
hrefs

[<a href="https://www.iana.org/domains/example">More information...</a>]

In [36]:
hrefs = soup.find_all('a', href='https://www.other-website.org/domains/example')
hrefs

[]

## Performance

In [37]:
%timeit re.findall(r'<p>([\w\W]*?)</p>', html)
%timeit tree.xpath('//p')
%timeit soup.find_all('p')

12.6 µs ± 663 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
15.9 µs ± 558 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
28.1 µs ± 1.47 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Мораль: regex -- это быстро.

BeautifulSoup чуть дольше lxml, т.к. он для удобства работы преобразует документ в некий внутренний формат -- собственно, суп.

## Если осталось время и хочется совсем живой практики: scrapy

Нужно минут 5-10, в зависимости от качества интернета.

А заодно попрактикуемся в ipython magic!

Будем парсить сайт с архивом курсов криптовалют: https://coinmarketcap.com/.

Хочется открыть [historical snapshots](https://coinmarketcap.com/historical/) и для каждой недели выгрузить полную табличку.

### Важно!

Совершенно не обязательно понимать, что будет происходить дальше.

Мораль: есть фреймворки, которые полностью берут на себя получение и парсинг HTML. 

А ещё они умеют распараллеливать процесс, вставлять случайные задержки (чтобы серверы не подумали, что вы робот и не заблокировали ваши запросы) и удобно экспортировать полученные данные (например, в базу данных). И многое другое. <br>И всё это контролируется `scrapy.cfg` -- одним конфигурационным файлом.

Вам остаётся только указать, какие куски текста надо выцеплять. И в какую базу класть. Ну, почти. :)

In [38]:
try:
    import scrapy
except:
    !pip install scrapy

In [39]:
!rm -rf coinmarketcap

In [40]:
!scrapy startproject coinmarketcap

New Scrapy project 'coinmarketcap', using template directory '/opt/conda/lib/python3.8/site-packages/scrapy/templates/project', created in:
    /home/jovyan/work/seminar-09/coinmarketcap

You can start your first spider with:
    cd coinmarketcap
    scrapy genspider example example.com


Что там внутри?

In [41]:
!cd coinmarketcap; ls -R

.:
coinmarketcap  scrapy.cfg

./coinmarketcap:
__init__.py  items.py  middlewares.py  pipelines.py  settings.py  spiders

./coinmarketcap/spiders:
__init__.py


Специальная функция, чтобы из ноутбука формировать код в проекте scrapy:

In [42]:
def dump_to(path):
    with open(path, 'w') as f:
        f.write(_i)  # _i это "последний выполненный Input" в iPython

### Внимание!

Следующие ячейки не будут выполняться. <br>Мы просто помещаем код в Input, а потом перекладываем его в нужный файл.

### Item: cперва определим, что хотим собирать

In [43]:
# -*- coding:utf8 -*-

import scrapy


class CurrencyItem(scrapy.Item):
    date = scrapy.Field()
    name = scrapy.Field()
    symbol = scrapy.Field()
    market_cap = scrapy.Field()
    price = scrapy.Field()

In [44]:
dump_to('./coinmarketcap/coinmarketcap/items.py')

### Spider: тот, кто собирает Item'ы

In [45]:
# -*- coding:utf8 -*-

from scrapy import Request
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
from scrapy.loader.processors import Join
from scrapy.loader import ItemLoader
from scrapy.selector import Selector
from coinmarketcap.items import CurrencyItem

class CurrencyLoader(ItemLoader):
    pass

class WeeklySpider(CrawlSpider):
    name = 'weekly'
    allowed_domains = ['coinmarketcap.com']
    start_urls = ['https://coinmarketcap.com/historical/']
    only_2018_april_regex = '/201904[0-9]{2}' # full history parsing takes ~4 hrs 

    rules = (
        Rule(LinkExtractor(allow=(only_2018_april_regex, )), callback='parse_weekly_report', follow=False),
    )

    def parse_weekly_report(self, response):
        
        hxs = Selector(response)
        items_html = hxs.xpath('//table//tr')
        #print(len(items_html),type(items_html),dir(items_html))
        #print(items_html)
        items = []
        
        item_names = items_html.xpath('//td[@class="cmc-table__cell cmc-table__cell--sticky cmc-table__cell--sortable cmc-table__cell--left cmc-table__cell--sort-by__name"]//div//a/text()').extract()
        item_symbols = items_html.xpath('//td[@class="cmc-table__cell cmc-table__cell--sortable cmc-table__cell--left cmc-table__cell--sort-by__symbol"]//div/text()').extract()
        item_caps = items_html.xpath('//td[@class="cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__market-cap"]//p/text()').extract()
        item_prices = items_html.xpath('//td[@class="cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__price"]//a/text()').extract()
        print(response.request.url)
        #print(len(item_prices),item_prices)
        
        for i in range(200):

            item = CurrencyItem()
            item['date'] = response.request.url.split('/')[-2]
            item['name'] = item_names[i]
            item['symbol'] = item_symbols[i]
            item['market_cap'] = item_caps[i]
            item['price'] = item_prices[i]

            yield item
          

ModuleNotFoundError: No module named 'coinmarketcap.items'

In [ ]:
dump_to('./coinmarketcap/coinmarketcap/spiders/weekly.py')

### Pipeline: например, экспорт в БД

Не бойтесь, про это будет отдельный семинар! (начиная с учебного года 2019-2020 - можно бояться)

In [ ]:
# -*- coding: utf-8 -*-

import os, logging
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, Table, Column, Integer, String, Date, MetaData, ForeignKey
from sqlalchemy.engine.url import URL
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import NullPool
from scrapy.exceptions import DropItem
from scrapy import signals
from coinmarketcap.items import CurrencyItem
import pandas as pd
from collections import Counter

logger = logging.getLogger(__name__)

DeclarativeBase = declarative_base()

class Currency(DeclarativeBase):
    __tablename__ = 'currency'
    __table_args__ = {'sqlite_autoincrement': True}

    id = Column('id', Integer, primary_key=True)
    date = Column('date', Date)
    name = Column('name', String)
    symbol = Column('symbol', String)
    market_cap = Column('market_cap', String)
    price = Column('price', String)

    def __init__(self, item):
        self.date = pd.to_datetime(item['date'], format='%Y%m%d')
        self.name = item['name']
        self.symbol = item['symbol']
        self.market_cap = item['market_cap']
        self.price = item['price']

    def __repr__(self):
        return "<Currency({0}, {1}, {2})>".format(self.id, self.symbol, self.market_cap)


class SqlitePipeline(object):
    def __init__(self, settings):
        self.database = settings.get('DATABASE')
        self.sessions = {}

    @classmethod
    def from_crawler(cls, crawler):
        pipeline = cls(crawler.settings)
        crawler.signals.connect(pipeline.spider_closed, signals.spider_closed)
        crawler.signals.connect(pipeline.spider_opened, signals.spider_opened)
        return pipeline

    def create_engine(self):
        engine = create_engine(URL(**self.database), poolclass=NullPool)
        return engine

    def create_tables(self, engine):
        DeclarativeBase.metadata.create_all(engine, checkfirst=True)

    def create_session(self, engine):
        session = sessionmaker(bind=engine)()
        return session

    def spider_opened(self, spider):
        engine = self.create_engine()
        self.create_tables(engine)
        session = self.create_session(engine)
        self.sessions[spider] = session

    def spider_closed(self, spider):
        session = self.sessions.pop(spider)
        session.close()

    def process_item(self, item, spider):
        session = self.sessions[spider]
        currency = Currency(item)
        link_exists = session.query(Currency).filter_by(symbol=item['symbol'], date=item['date']).first() is not None
        
        if link_exists:
            logger.info('Item {} is in db'.format(currency))
            return item

        try:
            session.add(currency)
            session.commit()
            logger.info('Item {} stored in db'.format(currency))
        except Exception as e:
            logger.info('Failed to add {} to db'.format(currency))
            session.rollback()
            raise e

        return item

In [ ]:
dump_to('./coinmarketcap/coinmarketcap/pipelines.py')

### Settings: общие настройки scrapy

In [ ]:
# -*- coding: utf-8 -*-

# Scrapy settings for coinmarketcap project
#
# For simplicity, this file contains only settings considered important or
# commonly used. You can find more settings consulting the documentation:
#
#     https://doc.scrapy.org/en/latest/topics/settings.html
#     https://doc.scrapy.org/en/latest/topics/downloader-middleware.html
#     https://doc.scrapy.org/en/latest/topics/spider-middleware.html

BOT_NAME = 'coinmarketcap'

SPIDER_MODULES = ['coinmarketcap.spiders']
NEWSPIDER_MODULE = 'coinmarketcap.spiders'

DATABASE = {
    'drivername': 'sqlite',
    # 'host': 'localhost',
    # 'port': '5432',
    # 'username': 'YOUR_USERNAME',
    # 'password': 'YOUR_PASSWORD',
    'database': 'weekly.sqlite'
}

# Crawl responsibly by identifying yourself (and your website) on the user-agent
USER_AGENT = '%s' % (BOT_NAME)

# Obey robots.txt rules
# ROBOTSTXT_OBEY = True

# Configure maximum concurrent requests performed by Scrapy (default: 16)
CONCURRENT_REQUESTS = 1

# Configure a delay for requests for the same website (default: 0)
# See https://doc.scrapy.org/en/latest/topics/settings.html#download-delay
# See also autothrottle settings and docs
#DOWNLOAD_DELAY = 3
# The download delay setting will honor only one of:
#CONCURRENT_REQUESTS_PER_DOMAIN = 16
#CONCURRENT_REQUESTS_PER_IP = 16

# Disable cookies (enabled by default)
#COOKIES_ENABLED = False

# Disable Telnet Console (enabled by default)
#TELNETCONSOLE_ENABLED = False

# Override the default request headers:
#DEFAULT_REQUEST_HEADERS = {
#   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#   'Accept-Language': 'en',
#}

# Enable or disable spider middlewares
# See https://doc.scrapy.org/en/latest/topics/spider-middleware.html
#SPIDER_MIDDLEWARES = {
#    'coinmarketcap.middlewares.CoinmarketcapSpiderMiddleware': 543,
#}

# Enable or disable downloader middlewares
# See https://doc.scrapy.org/en/latest/topics/downloader-middleware.html
#DOWNLOADER_MIDDLEWARES = {
#    'coinmarketcap.middlewares.CoinmarketcapDownloaderMiddleware': 543,
#}

# Enable or disable extensions
# See https://doc.scrapy.org/en/latest/topics/extensions.html
#EXTENSIONS = {
#    'scrapy.extensions.telnet.TelnetConsole': None,
#}

# Configure item pipelines
# See https://doc.scrapy.org/en/latest/topics/item-pipeline.html
ITEM_PIPELINES = {
   'coinmarketcap.pipelines.SqlitePipeline': 300,
}

# Enable and configure the AutoThrottle extension (disabled by default)
# See https://doc.scrapy.org/en/latest/topics/autothrottle.html
AUTOTHROTTLE_ENABLED = False
# The initial download delay
AUTOTHROTTLE_START_DELAY = 5
# The maximum download delay to be set in case of high latencies
AUTOTHROTTLE_MAX_DELAY = 60
# The average number of requests Scrapy should be sending in parallel to
# each remote server
#AUTOTHROTTLE_TARGET_CONCURRENCY = 1.0
# Enable showing throttling stats for every response received:
#AUTOTHROTTLE_DEBUG = False

# Enable and configure HTTP caching (disabled by default)
# See https://doc.scrapy.org/en/latest/topics/downloader-middleware.html#httpcache-middleware-settings
#HTTPCACHE_ENABLED = True
#HTTPCACHE_EXPIRATION_SECS = 0
#HTTPCACHE_DIR = 'httpcache'
#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'

LOG_FILE = 'crawling.log'

In [ ]:
dump_to('./coinmarketcap/coinmarketcap/settings.py')

Посмотрим на структуру scrapy проекта ещё раз

In [ ]:
!cd coinmarketcap; ls -R

### Всё! 

Запустим паука! 


In [ ]:
%%timeit -n 1 -r 1
!cd coinmarketcap ; scrapy crawl weekly

Посмотрим, что получилось:

In [ ]:
import sqlite3
import pandas as pd

connection = sqlite3.connect('./coinmarketcap/weekly.sqlite')

df = pd.read_sql_query("SELECT * FROM currency", connection)
print(df.shape)
df.head()